<a href="https://colab.research.google.com/github/szambetti/INVEST_IO/blob/master/stock_scraping_investing_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Questo notebook scarica il calendario economico da investing, le dati sono settabili come riportato sotto


In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from tqdm import tqdm_notebook as tqdm
import pandas
import json
import pprint

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

In [3]:
#importa la data correte
from datetime import date
today = date.today()
today_us_format = today.strftime("%m/%d/%Y")

In [23]:
#function per cliccare pulsanti su chrome .click() di selenium non funziona
def click_on_button_id(x):
  element = wd.find_element_by_id(x)
  wd.execute_script("arguments[0].click();", element)

In [37]:
#CALENDARIO ECONOMICO

url = 'https://www.investing.com/economic-calendar'
import time
import numpy as np

from selenium import webdriver
wd.set_window_size(1920,1080)
wd.maximize_window()
wd.get(url)

#apply filter on dates, scarica sempre da startdate a oggi, !!! FORMATO DATE US !!!
startdate = '08/01/2020'

click_on_button_id('datePickerToggleBtn')
click_on_button_id('startDate')
wd.save_screenshot('screen.png')
wd.find_element_by_id("startDate").clear()
wd.find_element_by_id("startDate").send_keys(startdate)
click_on_button_id('endDate')
wd.find_element_by_id("endDate").clear()
wd.find_element_by_id("endDate").send_keys(today_us_format)
click_on_button_id("applyBtn")
#wait for the apply button to be effective
time.sleep(5)

list_of_news = []

#launch main scraping loop
for tr in wd.find_elements_by_xpath('//*[@id="economicCalendarData"]/tbody//tr'):
    tds = tr.find_elements_by_tag_name('td')
    list_of_news_data=[]
    for td in tds:
      if td.text != '':
        list_of_news_data.append(td.text)
      elif td.get_attribute('title') != '':
        list_of_news_data.append(td.get_attribute('title'))
      else:
        list_of_news_data.append('')
    if len(list_of_news_data) == 1:
        date = list_of_news_data[0]
    elif len(list_of_news_data) == 4:
      list_of_news.append({'date' : date, 'time': list_of_news_data[0], 
                            'currency' : list_of_news_data[1],
                            'importance' : list_of_news_data[2], 
                            'event' : list_of_news_data[3],
                            'actual' : '',
                            'expected' : '',
                            'previous' : ''})
    else:
      list_of_news.append({'date' : date, 'time': list_of_news_data[0], 
                            'currency' : list_of_news_data[1],
                            'importance' : list_of_news_data[2], 
                            'event' : list_of_news_data[3],
                            'actual' : list_of_news_data[4],
                            'expected' : list_of_news_data[5],
                            'previous' : list_of_news_data[6]})

In [40]:
from pandas import DataFrame
df = DataFrame (list_of_news)
df.head()
df.to_csv('sr_economic_calendar.csv')

In [35]:
df.head(15)

,date,time,currency,importance,event,actual,expected,previous
0,"Sunday, August 9, 2020",20:00,SGD,Low Volatility Expected,GDP (YoY) (Q2),,,-12.6%
1,"Sunday, August 9, 2020",20:00,SGD,Low Volatility Expected,GDP (QoQ) (Q2),,,-41.2%
2,"Sunday, August 9, 2020",21:00,NZD,Moderate Volatility Expected,ANZ Business Confidence (Jul),,,-34.4
3,"Sunday, August 9, 2020",21:00,NZD,Low Volatility Expected,NBNZ Own Activity (Jul),,,-25.9%
4,"Sunday, August 9, 2020",21:30,CNY,Moderate Volatility Expected,CPI (MoM) (Jul),,0.4%,-0.1%
5,"Sunday, August 9, 2020",21:30,CNY,Moderate Volatility Expected,CPI (YoY) (Jul),,2.6%,2.5%
6,"Sunday, August 9, 2020",21:30,CNY,Moderate Volatility Expected,PPI (YoY) (Jul),,-2.5%,-3.0%
